In [48]:
# import webget
import pandas as pd
import numpy as np
import tensorflow as tf

# webget.download('https://archive.ics.uci.edu/ml/machine-learning-databases/poker/poker-hand-testing.data')
# webget.download('https://archive.ics.uci.edu/ml/machine-learning-databases/poker/poker-hand-training-true.data')

train_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/poker/poker-hand-training-true.data'
test_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/poker/poker-hand-testing.data'

col_names = {0:'Suit of card #1', 1:'Rank of card #1',
             2:'Suit of card #2', 3:'Rank of card #2',
             4:'Suit of card #3', 5:'Rank of card #3',
             6:'Suit of card #4', 7:'Rank of card #4',
             8:'Suit of card #5', 9:'Rank of card #5',
             10:'Poker Hand'}

# train = pd.read_csv('poker-hand-training-true.data', header=None)
train = pd.read_csv(train_url, header=None)
train.rename(col_names, axis='columns', inplace=True)

# test = pd.read_csv('poker-hand-testing.data', header=None)
test = pd.read_csv(test_url, header=None)
test.rename(col_names, axis='columns', inplace=True)

x_train = train.iloc[:,0:10]
y_train = train.iloc[:,10]
x_test = train.iloc[:,0:10]
y_test = train.iloc[:,10]

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(10,)))
model.add(tf.keras.layers.Dense(2048, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train.values, y_train.values, epochs=1000)
model.evaluate(x_test.values, y_test.values)

In [ ]:
model.save('my_model.h5')

In [55]:
new_model = tf.keras.models.load_model('my_model2.h5')
new_model.summary()
loss, acc = new_model.evaluate(x_test.values, y_test.values)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 10)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 2048)              22528     
_________________________________________________________________
dense_5 (Dense)              (None, 10)                20490     
Total params: 43,018
Trainable params: 43,018
Non-trainable params: 0
_________________________________________________________________
25010/25010 [==============================] - 3s 128us/sample - loss: 6.8475e-04 - acc: 1.0000
Restored model, accuracy: 100.00%


In [62]:
predictions = new_model.predict_proba(x_test)
    vals = ['Nothing', 'One Pair', 'Two Pairs', 'Three of a kind', 'Straight',
            'Flush', 'Full house', 'Four of a kind', 'Straight flush', 'Royal flush']
count = 0
fails = []
for x in range(len(predictions)):
    pred = np.argmax(predictions[x])
    real = y_test.loc[x]
    if pred == 9:
        pass
    if pred == real:
        count += 1
    else:
        print('prediction: ' + vals[pred], 'real: ' + vals[real])
        fails.append((pred,real))

# for x in sorted(list(set(fails)), key=lambda x: x[0]):
#     print('prediction: ' + vals[x[0]], 'real: ' + vals[x[1]])
print('acc', count/len(predictions))

acc 1.0


In [69]:
import random
test = [[random.choice(range(1,5)),
  random.choice(range(1,14)),
  random.choice(range(1,5)),
  random.choice(range(1,14)),
  random.choice(range(1,5)),
  random.choice(range(1,14)),
  random.choice(range(1,5)),
  random.choice(range(1,14)),
  random.choice(range(1,5)),
  random.choice(range(1,14))] for _ in range(100)]

arr = [[1,10,1,11,1,13,1,12,1,1],
       [1,1,1,13,1,12,1,11,1,10],
       [1,13,1,12,1,11,1,10,1,9],
       [1,1,2,1,4,6,3,6,1,13],
       [2,9,4,9,1,13,1,5,2,13]]
# predi = new_model.predict_proba([arr])
predi = new_model.predict_proba([test])
# print(predi)
arr2 = [x * 100 // 1 for x in predi]
# print(arr2)
for x in range(len(predi)):
    print(test[x],vals[np.argmax(predi[x])])

[3, 5, 2, 7, 2, 11, 2, 11, 3, 11] Two Pairs
[3, 10, 4, 8, 3, 6, 3, 6, 1, 5] One Pair
[3, 10, 2, 4, 4, 5, 3, 2, 2, 1] Nothing
[4, 11, 4, 1, 1, 5, 2, 9, 4, 9] One Pair
[3, 3, 3, 5, 2, 13, 4, 4, 3, 4] One Pair
[2, 10, 1, 7, 1, 11, 4, 6, 4, 8] Nothing
[2, 9, 2, 9, 4, 11, 3, 4, 1, 1] One Pair
[3, 7, 2, 3, 2, 4, 4, 7, 2, 7] Three of a kind
[2, 11, 1, 1, 4, 3, 2, 1, 4, 6] One Pair
[3, 13, 2, 10, 4, 2, 4, 2, 3, 6] One Pair
[1, 2, 3, 13, 1, 8, 4, 13, 4, 6] One Pair
[4, 11, 4, 6, 3, 2, 2, 4, 3, 9] Nothing
[4, 7, 2, 4, 2, 12, 4, 12, 4, 1] One Pair
[4, 10, 2, 13, 2, 10, 3, 9, 2, 8] One Pair
[3, 3, 1, 1, 1, 10, 2, 13, 4, 6] Nothing
[2, 3, 3, 12, 2, 11, 2, 10, 2, 3] One Pair
[3, 5, 3, 3, 3, 4, 3, 7, 4, 1] Nothing
[1, 10, 4, 11, 3, 11, 1, 8, 3, 13] One Pair
[2, 12, 3, 1, 2, 8, 3, 11, 3, 12] One Pair
[1, 6, 1, 9, 1, 10, 1, 4, 2, 12] Nothing
[1, 8, 1, 12, 4, 3, 1, 9, 4, 6] Nothing
[3, 2, 3, 6, 3, 11, 4, 4, 4, 9] Nothing
[2, 5, 4, 1, 2, 6, 1, 8, 4, 1] One Pair
[2, 10, 3, 8, 1, 8, 3, 11, 4, 8] Three of a

In [ ]:
arr = [[1,1,1,2,1,3,1,4,1,5]]

def check_straight(ranks):
    nums = range(10)
    sorted_ranks = sorted(ranks)
    return nums[(sorted_ranks[0]):(len(sorted_ranks)+sorted_ranks[0])] == sorted_ranks or set(ranks) == {1,10,11,12,13}

def check_flush(suits):
    return len(set(suits)) == 1

def check_four_of_a_kind(ranks):

def check_three_of_a_kind(hand):
    
def check_two_pairs(hand):
    
def check_pair(hand):
    
def evaluate_hand(hand):
    s1, r1, s2, r2, s3, r3, s4, r4, s5, r5 = hand[0]
    suits = [s1,s2,s3,s4,s5]
    ranks = [r1,r2,r3,r4,r5]
    if check_flush(suits) and check_straight(ranks) and set(ranks) == {1,10,11,12,13}:
        return 9
    elif check_flush(suits) and check_straight(ranks):
        return 8
    elif check_four_of_a_kind(ranks):
        return 7
    elif check_pair()[0] and check_three_of_a_kind()[0] and check_pair()[1] != check_three_of_a_kind()[1]:
        return 6
    elif check_flush(suits):
        return 5
    elif check_straight(ranks):
        return 4
    elif check_three_of_a_kind(hand)[0]:
        return 3
    elif check_two_pairs(hand):
        return 2
    elif check_pair(hand)[0]:
        return 1
    else:
        return 0